In [1]:
!pip install xlsxwriter
!pip install openpyxl

﻿
[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\rami8629\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


﻿
[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\rami8629\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [52]:
from arcgis.gis import GIS

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

from openpyxl import Workbook, load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

import os

In [81]:
class ArcGISDataAnalyzer:
    def __init__(self, target_date, excel_dir):
        self.target_date = datetime.strptime(target_date, "%Y-%m-%d").date()
        self.excel_dir = excel_dir
        self.gis = GIS("Pro")
        self.items_before_date_all_users_df = None
        self.sorted_user_details = None
        
        self.sorted_user_details = None
        
    # Define a function to calculate credits per month based on item type and size
    def _calculate_credits_per_month(self, item_type, item_size):
        if item_type == 'Feature Service':
            credits_per_10mb = 2.4
            credits_per_month = (item_size / 10) * credits_per_10mb
        elif item_type == 'Tiled Imagery':
            credits_per_gb = 1.2
            credits_per_month = (item_size / 1024) * credits_per_gb
        elif item_type == 'Dynamic Imagery':
            num_images = 100  # Example number of images
            if num_images <= 10:
                credits_per_day = 10
            elif num_images <= 100:
                credits_per_day = 20
            elif num_images <= 1000:
                credits_per_day = 40
            elif num_images <= 10000:
                credits_per_day = 80
            elif num_images <= 100000:
                credits_per_day = 160
            else:
                credits_per_day = 320
            credits_per_month = (credits_per_day * 30)  # Assuming 30 days in a month
        else:
            credits_per_month = (item_size / 10) * 2.4  # Default value if item type is not found
        return credits_per_month
    
    def _get_all_items(self, user):
        users_folder_list = user.folders
        user_items = user.items()
        
        for user_folder in users_folder_list:
            folder_items = user.items(folder=user_folder['title'])
            user_items.extend(folder_items)
        return (user_items)


    def get_items_before_date_all_users(self):
        gis = self.gis
        # Retrieve all users in the organization
        all_users = gis.users.search(query="*")  # You can modify the query as needed

        # Create a list to store item information
        all_items = []

        # Iterate through each user and retrieve their items
        for user in all_users:
            user_items = self._get_all_items(user)
           
            for item in user_items:
                # Convert timestamp to datetime
                last_viewed = item.lastViewed
                if last_viewed == -1:
                    last_viewed = datetime.utcfromtimestamp(0)  # Set to epoch time
                else:
                    last_viewed = datetime.fromtimestamp(last_viewed / 1000)  # Convert milliseconds to seconds

                # Convert datetime to ArcGIS Online format date (e.g., "yyyy-mm-ddTHH:mm:ssZ")
                last_viewed_agol_format = last_viewed.strftime("%Y-%m-%d")

                if last_viewed.date() <= self.target_date:
                    # Calculate credits per month based on item type and size
                    item_type = item.type  # Assuming the 'type' attribute is present in the item object
                    item_size = item.size / (1024 * 1024)  # Convert bytes to megabytes
                    credits_per_month = self._calculate_credits_per_month(item_type, item_size)
                    item_url = f"{gis.url}home/item.html?id={item.id}"
                    item_info = {
                        "item_name": item.title,
                        "item_id": item.itemid,
                        "item_url": item_url,
                        "last_viewed_date": last_viewed_agol_format,
                        "username": user.username,
                        "full_name": user.fullName,
                        "email": user.email,
                        "item_size_mb": item_size,
                        "item_type": item_type,
                        "estimated_credits_per_month": credits_per_month,
                        "estimated_cost_per_month": credits_per_month * 0.1,
                        "keep_item_yes_no": "No"
                    }
                    all_items.append(item_info)

        # Convert the list of items to a pandas DataFrame
        items_df = pd.DataFrame(all_items)

        del gis
        return items_df

    def get_sorted_user_details(self, items_df):
        gis = self.gis        
        items_before_date_all_users_df = items_df
        # Extract unique usernames
        unique_usernames = items_before_date_all_users_df['username'].unique()

        # Group the DataFrame by 'username' and calculate the total counts of items owned by each user
        user_item_counts = items_before_date_all_users_df.groupby('username')['item_id'].count()

        # Group the DataFrame by 'username' and sum the estimated credits per month, item size, and estimated cost per month for each user
        user_aggregated_data = items_before_date_all_users_df.groupby('username').agg({
            'estimated_credits_per_month': 'sum',
            'item_size_mb': 'sum',
            'estimated_cost_per_month': 'sum'
        })

        # Create a list to store user details including last login date, full name, email, item counts, total credit cost, total item size, and total cost per month
        user_details_list = []

        # Iterate through unique usernames and query the last login date, full name, email address, item counts, total credit cost, total item size, and total cost per month for each user
        for username in unique_usernames:
            user = gis.users.get(username)
            last_login_date_str = (datetime.fromtimestamp(user.lastLogin / 1000)).strftime("%Y-%m-%d")
            item_count = user_item_counts.get(username, 0)
            total_credits = user_aggregated_data.loc[username, 'estimated_credits_per_month']
            total_item_size_mb = user_aggregated_data.loc[username, 'item_size_mb']
            total_cost_per_month = user_aggregated_data.loc[username, 'estimated_cost_per_month']
            user_details = {
                "username": username,
                "last_login_date": last_login_date_str,
                "full_name": user.fullName,
                "email": user.email,
                "item_count": item_count,
                "total_credits": total_credits,
                "total_item_size_mb": total_item_size_mb,
                "total_cost_per_month": total_cost_per_month
            }
            user_details_list.append(user_details)

        # Sort the user details list by highest total cost per month
        sorted_user_details = sorted(user_details_list, key=lambda x: x['total_cost_per_month'], reverse=True)

        return pd.DataFrame(sorted_user_details)
    
    def get_users_before_target_date(self):
        gis = self.gis
        # Initialize the GIS instance
        all_users = gis.users.search(query="*")  # You can modify the query as needed
        
        adjusted_target_date = self.target_date  - pd.DateOffset(months=6)
        usernames_before_date = []

        for username in all_users:
            if username.lastLogin == -1:
                last_login_date = datetime.utcfromtimestamp(0)  # Set to epoch time
            else:
                last_login_date = datetime.fromtimestamp(username.lastLogin/1000)
            try:
                user_items = username.items()
                item_count = len(user_items)
            except Exception as e:
                # Handle the exception when the user has no items
                print(f"Error retrieving items for user {username.username}: {str(e)}")
                item_count = 0
            # Check if the last_login_date is before the adjusted_target_date
            if last_login_date < adjusted_target_date:
                user_details = {
                    "username": username.username,
                    "last_login_date": last_login_date,
                    "full_name": username.fullName,
                    "email": username.email,
                    "item_count": item_count,
                    "keep_user_account_yes_no": "No"
                }
                usernames_before_date.append(user_details)
        
        return pd.DataFrame(usernames_before_date)

In [88]:
# Create an instance of the ArcGISDataAnalyzer class with target date "2023-01-01"
target_date = "2023-01-01"
excel_dir = r"D:\OneDrive - Esri\Demos & Blogs\Code-Repo\Notebooks"

In [89]:
analyzer = ArcGISDataAnalyzer(target_date, excel_dir)

In [90]:
# Call the get_items_before_date_all_users() method on the analyzer instance
df1 = analyzer.get_items_before_date_all_users()
df1.head()

,item_name,item_id,item_url,last_viewed_date,username,full_name,email,item_size_mb,item_type,estimated_credits_per_month,estimated_cost_per_month,keep_item_yes_no
0,CBC Rare Bird Observation Editor,8832465d17a742b4bb1e3aaf7556ad1c,https://www.arcgis.com/home/item.html?id=88324...,1970-01-01,AGiron_aid,Amanda Giron,AGiron@esri.com,0.000000,Code Attachment,0.000000,0.000000,No
1,Christmas Bird Count Rare Bird Reviewer Experi...,538febabfcc84ac2bd82a6606e21ee8c,https://www.arcgis.com/home/item.html?id=538fe...,2022-03-22,AGiron_aid,Amanda Giron,AGiron@esri.com,0.012760,Web Experience,0.003062,0.000306,No
2,Christmas Bird Count Rare Bird Reviewer Map,8998185999714d7dbdd36517c9516567,https://www.arcgis.com/home/item.html?id=89981...,2022-04-07,AGiron_aid,Amanda Giron,AGiron@esri.com,0.019292,Web Map,0.004630,0.000463,No
3,Christmas Bird Count Rare Bird Attachment View...,279f32030673429981a1b13bfe255721,https://www.arcgis.com/home/item.html?id=279f3...,2022-04-07,AGiron_aid,Amanda Giron,AGiron@esri.com,0.000591,Web Mapping Application,0.000142,0.000014,No
4,World_Giving_Index_WFL1,cd31a4a97d5041de80c72667de15addc,https://www.arcgis.com/home/item.html?id=cd31a...,1970-01-01,AidDevService,Aid & Development,apfister@esri.com,12.999071,Service Definition,3.119777,0.311978,No


In [91]:
df2 = analyzer.get_sorted_user_details(df1)
df2.head()

,username,last_login_date,full_name,email,item_count,total_credits,total_item_size_mb,total_cost_per_month
0,boneill.aid,2023-08-25,Brendan O'Neill,boneill@esri.com,620,3707.555538,15448.148077,370.755554
1,kmorrish_dev,2023-08-24,Keera Morrish,kmorrish@esri.com,334,1471.543300,6131.430416,147.154330
2,ralouta.aiddev,2023-08-25,Rami Alouta,ralouta@esri.com,259,981.477427,4089.489280,98.147743
3,dgadsden_aid2,2022-12-01,David Gadsden,dgadsden@esri.com,77,488.466827,2035.278446,48.846683
4,jyoon.aid,2022-12-14,Juhan Yoon,jyoon@esri.com,11,433.192369,1804.968205,43.319237


In [92]:
df3 = analyzer.get_users_before_target_date()
df3.head()

,username,last_login_date,full_name,email,item_count,keep_user_account_yes_no
0,aiddev_admin,2015-06-25 15:06:57,Aid & Development Admin,apfister@esri.com,0,No
1,apfister,2022-04-19 23:39:55,d1a0422a-1c30-4e9a-b870-d2739edbf9a2,apfister@esri.com,0,No
2,bladds_EsriAidDev,2019-12-11 16:20:02,Bryan Ladds,bladds@esri.ca,0,No
3,bmccardle_aid,2021-10-01 16:00:54,Brianna McCardle,bmccardle@esri.com,15,No
4,boneill.geong,2019-01-16 22:15:07,Brendan O'Neill,boneill@esri.com,0,No


In [39]:
# Load the Excel file
file_path = analyzer.add_yes_no_column_to_excel(df1, df2, df3)
file_path

'D:\\OneDrive - Esri\\Demos & Blogs\\ArcGIS Resources\\ArcGIS Admin\\ArcGIS_Online_Management.xlsx'

In [27]:
mailingList = set(pd.concat([df2['email'], df3['email']]))
print(str(mailingList).replace("'", "").replace(",",";"))

{MDONOVAN@iadb.org; JSullivan@esri.com; cmenzel@esri.com; Firasa@geoinfo.dk; jbayer@esri.com; ckiefer@esri.com; ckwon@esri.com; lndungo@esriea.com; tbutcher@esri.com; jlaws@esri.com; ralouta@esri.com; rdonihue@esri.com; jvandeusen@esri.com; mburger@esri-southafrica.com; cbrigham@esri.com; RLehman@esri.com; VCoffin@esri.com; mmusgrave@esri.com; croland@esri.com; JVaughan-Gibson@esri.com; MSeybert@esri.com; jmunyao@esri.com; slunderville@essex.org; Adam_Jenkins@esri.com; hturvene@esri.com; RGreenebaum@esri.com; jhughes@esri.ca; rbilly@esri.com; khess@esri.com; dgadsden@esri.com; AGiron@esri.com; akim@esri.com; apfister@esri.com; ksmyth@esri.com; ssawaya@esri.com; phornstein@esri.com; j.bell@esri.com; slibby@esri.com; brollison@esri.com; amakowicki@esri.com; cmain@esri.com; c.despierre-corporon@esri.ch; bstayer@esri.com; JChavez@esri.com; KHuber-Wilker@esri.com; niklasg@geoinfo.dk; linxuelin@esrichina.com.cn; oiannone@esri.com; bmccardle@esri.com; cmarietta@esri.com; speluso@esri.com; dko

In [149]:

#         elif sheet_name == 'inactive_items_per_user':
#             # Delete all rows
#             df = pd.DataFrame()
#             # Map it to a predefined df2
#             new_rows = df2
#         elif sheet_name == 'inactive_users':
#             # Use 'username' as the key to compare
#             existing_values = set(df['username'].values)
#             # Filter the data for dictionaries with no matching values
#             values_to_keep = [d for d in df3['username'] if d not in existing_values]
#             # Append the new dictionaries to the DataFrame
#             new_rows = df3[df3['username'].isin(values_to_keep)]

In [209]:
class test_excel:
    def __init__(self, df1, df2, df3, excel_dir):
        self.df1 = df1
        self.df2 = df2
        self.df3 = df3
        self.sheet_names = ['inactive_items']
        self.key = 'item_id'
        self.df_dict = None
        self.df = None
        
        self.file_name = "ArcGIS_Online_Management.xlsx"
        self.file_path = os.path.join(excel_dir, self.file_name)
    
    def create_update_excel(self):
        # Check if the path exists
        if os.path.exists(self.file_path):
            # If it exists, run function1
            self._excel_updater()
        else:
            # If it doesn't exist, run function2
            self._excel_creator()
    def _excel_creator(self):

        with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
            self.df1.to_excel(writer, sheet_name='inactive_items')
            self.df2.to_excel(writer, sheet_name='inactive_items_per_user')
            self.df3.to_excel(writer, sheet_name=f'inactive_users')

        # Load the workbook using openpyxl
        workbook = load_workbook(self.file_path)
        sheet_name = f'inactive_users'
        sheet = workbook[sheet_name]

        # Define the Yes/No choices
        choices = ["Yes", "No"]

        # Define the data validation rule
        dv = DataValidation(type="list", formula1=f'"{",".join(choices)}"')

        # Apply the data validation to column G
        col_letter = 'G'
        for row in range(1, sheet.max_row + 1):
            dv.add(sheet[f'{col_letter}{row}'])

        # Add the data validation rule to the worksheet
        sheet.add_data_validation(dv)

        # Save the updated Excel file
        workbook.save(self.file_path)

        return self.file_path

    def _excel_updater(self):
        # Load the existing data from the Excel file
        book = load_workbook(self.file_path)
        # Use pd.ExcelFile to read all sheets
        with pd.ExcelFile(self.file_path, engine='openpyxl') as xls:
            self.df_dict = pd.read_excel(xls, sheet_name=None)
        for sheet in self.sheet_names:
            if sheet== 'inactive_items':
                if sheet in self.df_dict:
                    self.df = self.df_dict[sheet]
                else:
                    self.df = pd.DataFrame()

                # Create a set of existing values for the specified column for quick lookup
                existing_values = set(self.df[self.key].values)

                # Filter the data for dictionaries with no matching values
                values_to_keep = [d for d in df1[self.key] if d not in existing_values]

                # Append the new dictionaries to the DataFrame
                new_rows = df1[df1[self.key].isin(values_to_keep)]
                self.df = self.df.append(new_rows, ignore_index=True)

                with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
                    writer.book = book
                    writer.sheets = {ws.title: ws for ws in book.worksheets}

                    for sheet_df, frame in self.df_dict.items():
                        if sheet_df != sheet:
                            frame.to_excel(writer, sheet_name=sheet, index=False)
                    self.df.to_excel(writer, sheet_name=sheet, index=False)


In [210]:
updater = test_excel(df1, df2, df3, excel_dir)

In [214]:
updater.create_update_excel()

[209]:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
